In [14]:
import pandas as pd

# cc = pd.read_csv("Data/coaches_career.txt")
# cs = pd.read_csv("Data/coaches_season.txt")
# # draft = pd.read_csv("Data/draft.txt")
# pa = pd.read_csv("Data/player_allstar.txt")
# pp = pd.read_csv("Data/player_playoffs.txt")
# ppc = pd.read_csv("Data/player_playoffs_career.txt")
# prs = pd.read_csv("Data/player_regular_season.txt")
# prsc = pd.read_csv("Data/player_regular_season_career.txt")
# player = pd.read_csv("Data/players.txt")
# ts = pd.read_csv("Data/team_season.txt")
# team = pd.read_csv("Data/teams.txt")
# pbp = pd.read_csv("Kaggle data/play_by_play.csv", nrows=1000)

pd.set_option('display.max_rows', None)


In [20]:
d1 = pd.read_csv("Kaggle Data/game.csv").sort_values(by="game_date")
d1.iloc[1100:1200]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
1100,21949,1610610024,BAL,Baltimore Bullets,24900161,1949-12-10 00:00:00,BAL vs. BOM,W,0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,66.0,-3,0,Regular Season
1101,21949,1610612747,MNL,Minneapolis Lakers,24900159,1949-12-10 00:00:00,MNL vs. DN,W,0,32.0,...,NaN,NaN,NaN,NaN,NaN,28.0,76.0,-13,0,Regular Season
1102,21949,1610612738,BOS,Boston Celtics,24900158,1949-12-10 00:00:00,BOS vs. SHE,W,0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,78.0,-5,0,Regular Season
1103,21949,1610612758,ROC,Rochester Royals,24900162,1949-12-10 00:00:00,ROC vs. SYR,W,0,18.0,...,NaN,NaN,NaN,NaN,NaN,41.0,63.0,-6,0,Regular Season
1104,21949,1610610036,WAS,Washington Capitols,24900160,1949-12-10 00:00:00,WAS vs. FTW,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,69.0,-6,0,Regular Season
1109,21949,1610612755,SYR,Syracuse Nationals,24900166,1949-12-11 00:00:00,SYR vs. SHE,W,0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,72.0,-17,0,Regular Season
1108,21949,1610612747,MNL,Minneapolis Lakers,24900164,1949-12-11 00:00:00,MNL vs. PHW,W,0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,81.0,-9,0,Regular Season
1105,21949,1610612737,TCB,Tri-Cities Blackhawks,24900167,1949-12-11 00:00:00,TCB vs. DN,W,0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,66.0,-32,0,Regular Season
1106,21949,1610612765,FTW,Ft. Wayne Zollner Pistons,24900165,1949-12-11 00:00:00,FTW vs. INO,W,0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,74.0,-16,0,Regular Season
1107,21949,1610610037,WAT,Waterloo Hawks,24900163,1949-12-11 00:00:00,WAT vs. AND,L,0,27.0,...,NaN,NaN,NaN,NaN,NaN,35.0,87.0,4,0,Regular Season


In [27]:
d = []
for df in pd.read_csv('Kaggle Data/play_by_play.csv', chunksize=100000):
    d.append(df[df["game_id"] == 1610610023])
    break
d = pd.DataFrame(d[0])
d.head()

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag


In [6]:
def combinePlayerTeam(row, playerCol, teamCol):
    return [str(row[playerCol][i])+"_"+str(row[teamCol][i]) for i in range(0,len(row[teamCol]))]

def getPlayerInGame(pbp):
    df = pbp.groupby('game_id').agg({'player1_id': list, 'player1_team_abbreviation': list, 'player2_id': list, 'player2_team_abbreviation': list, 'player3_id': list, 'player3_team_abbreviation': list})
    df["combined"] = df.apply(lambda x: list(set(combinePlayerTeam(x, "player1_id", "player1_team_abbreviation") + combinePlayerTeam(x, "player2_id", "player2_team_abbreviation") +combinePlayerTeam(x, "player3_id", "player3_team_abbreviation"))), axis=1)
    df = df.explode("combined")
    df["player_id"] = df.apply(lambda x: x["combined"].split("_")[0], axis = 1)
    df["team_id"] = df.apply(lambda x: x["combined"].split("_")[1], axis = 1)

    return df[["player_id","team_id"]]

gameToPlayerChunks = []
count = 1
for df in pd.read_csv('Kaggle Data/play_by_play.csv', chunksize=100000):
    # if count % 100 == 0:
    #     chunk = pd.concat(gameToPlayerChunks)
    #     chunk.to_csv("Chunks/chunk_"+str(count/100)+".csv")
    # count+=1
    gameToPlayerChunks.append(getPlayerInGame(df))
gameToPlayer = pd.concat(gameToPlayerChunks)

In [8]:
# gameToPlayer.dropna(subset=["player_id"], inplace=True)
# gameToPlayer = gameToPlayer[gameToPlayer['player_id'] != 0]
gameToPlayer.to_csv("gameToPlayer.csv")

In [ ]:
gameToPlayer

In [ ]:
pp

In [ ]:
prsc

In [ ]:
cc.columns

In [ ]:
print(len(prsc.ilkid.unique()))
print(len(ppc.ilkid.unique()))

In [ ]:
d={}
a = []
for _, row in ppc.iterrows():
  if row.ilkid not in d:
    d[row.ilkid] = row
  else:
    a.append(row)
    a.append(d[row.ilkid])
df = pd.DataFrame(a).sort_values('ilkid')
df[df['leag'] == 'N']

In [ ]:
ts

In [ ]:
ts

In [ ]:
import numpy as np
currTeam = "LAL"
y = ts[(ts["team"] == currTeam) & (ts["year"] < 2004) & (ts["year"] > 1950) & (ts["leag"] == 'N')]
print("won:"+str(np.sum(y["won"])))
print("lost:"+str(np.sum(y["lost"])))

In [ ]:
game = pd.read_csv("Kaggle Data/game.csv")
game["year"] = game.apply(lambda x: int(x["game_date"][0:4]), axis = 1)
game = game[(game["year"] < 2004) & (game["year"] > 1950)]

In [ ]:
x = game[((game["team_abbreviation_home"] == currTeam) & (game["wl_home"] == "W")) | ((game["team_abbreviation_away"] == currTeam) & (game["wl_away"] == "W"))]
z = game[((game["team_abbreviation_home"] == currTeam) & (game["wl_home"] == "L")) | ((game["team_abbreviation_away"] == currTeam) & (game["wl_away"] == "L"))]

print("w:"+str(x.shape[0]))
print("z:"+str(z.shape[0]))


In [ ]:
game